# IS319 - Deep Learning

## TP3 - Recurrent neural networks

Credits: Andrej Karpathy

The goal of this TP is to experiment with recurrent neural networks for a character-level language model to generate text that looks like training text data.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 1. Text data preprocessing

Several text datasets are provided, feel free to experiment with different ones throughout the TP. At the beginning, use a small subset of a given dataset (for example use only 10k characters).

In [2]:
text_data_fname = 'baudelaire.txt'  # ~0.1m characters (French)
# text_data_fname = 'proust.txt'      # ~7.3m characters (French)
# text_data_fname = 'shakespeare.txt' # ~0.1m characters (English)
# text_data_fname = 'lotr.txt'        # ~2.5m characters (English)
# text_data_fname = 'doom.c'          # ~1m characters (C Code)
# text_data_fname = 'linux.c'         # ~11.5m characters (C code)

text_data = open(text_data_fname, 'r').read()
text_data = text_data[:10000] # use a small subset
print(f'Dataset `{text_data_fname}` contains {len(text_data)} characters.')
print('Excerpt of the dataset:')
print(text_data[:2000])

Dataset `baudelaire.txt` contains 10000 characters.
Excerpt of the dataset:
LES FLEURS DU MAL

par

CHARLES BAUDELAIRE


AU LECTEUR


La sottise, l'erreur, le péché, la lésine,
Occupent nos esprits et travaillent nos corps,
Et nous alimentons nos aimables remords,
Comme les mendiants nourrissent leur vermine.

Nos péchés sont têtus, nos repentirs sont lâches,
Nous nous faisons payer grassement nos aveux,
Et nous rentrons gaîment dans le chemin bourbeux,
Croyant par de vils pleurs laver toutes nos taches.

Sur l'oreiller du mal c'est Satan Trismégiste
Qui berce longuement notre esprit enchanté,
Et le riche métal de notre volonté
Est tout vaporisé par ce savant chimiste.

C'est le Diable qui tient les fils qui nous remuent!
Aux objets répugnants nous trouvons des appas;
Chaque jour vers l'Enfer nous descendons d'un pas,
Sans horreur, à travers des ténèbres qui puent.

Ainsi qu'un débauché pauvre qui baise et mange
Le sein martyrisé d'une antique catin,
Nous volons au passage un plaisir c

**(Question)** Create a character-level vocabulary for your text data. Create two dictionaries: `ctoi` mapping each character to an index, and the reverse `itoc` mapping each index to its corresponding character. Implement the functions to convert text to tensor and tensor to text using these mappings. Apply these functions to some text data.

In [3]:
# Create the vocabulary and the two mapping dictionaries
# YOUR CODE HERE

import numpy as np

vocabulary = list(set(text_data))

print(vocabulary)

ctoi = {vocabulary[num]: num for num in range(0, len(vocabulary))}

itoc = {num : vocabulary[num] for num in range(0, len(vocabulary))}

print(itoc)
print(ctoi)

# Implement the function converting text to tensor
def text_to_tensor(text, ctoi):
    # YOUR CODE HERE
    tensor = np.array([ctoi[c] for c in text])
    tensor = torch.Tensor(tensor).int()

    return tensor 

# Implement the function converting tensor to text
def tensor_to_text(tensor, itoc):
    # YOUR CODE HERE
    tensor = tensor.numpy()
    text = [itoc[i] for i in tensor]

    return text

# Apply your functions to some text data
# YOUR CODE HERE
sample = text_data[:10]

print("Sample : " + str(sample))

s_tensor = text_to_tensor(sample, ctoi)
s_text = tensor_to_text(s_tensor, itoc)

print("Tensor of sample : " + str(s_tensor))
print("Text of tensor : " + "".join(s_text))

['f', ':', 'M', 'ê', 'd', 'I', 'â', '-', 'à', 'x', '?', ' ', 'U', "'", 'g', 'É', '_', 'û', 'R', 'c', 'J', 'z', 's', 'N', 'F', 'è', '«', 'p', 'C', 'a', 'v', 'o', ';', 'h', 'A', 'r', 'u', 'E', 'k', 'O', 'B', 'l', 'b', '!', '»', 'V', ',', 'ô', 'P', 'y', 'é', 'm', '\n', 'S', 'j', 'W', 'G', 'T', 'n', 'ù', 'ç', 'e', 'D', 'H', '.', 'L', 't', 'Q', 'q', 'i', 'î']
{0: 'f', 1: ':', 2: 'M', 3: 'ê', 4: 'd', 5: 'I', 6: 'â', 7: '-', 8: 'à', 9: 'x', 10: '?', 11: ' ', 12: 'U', 13: "'", 14: 'g', 15: 'É', 16: '_', 17: 'û', 18: 'R', 19: 'c', 20: 'J', 21: 'z', 22: 's', 23: 'N', 24: 'F', 25: 'è', 26: '«', 27: 'p', 28: 'C', 29: 'a', 30: 'v', 31: 'o', 32: ';', 33: 'h', 34: 'A', 35: 'r', 36: 'u', 37: 'E', 38: 'k', 39: 'O', 40: 'B', 41: 'l', 42: 'b', 43: '!', 44: '»', 45: 'V', 46: ',', 47: 'ô', 48: 'P', 49: 'y', 50: 'é', 51: 'm', 52: '\n', 53: 'S', 54: 'j', 55: 'W', 56: 'G', 57: 'T', 58: 'n', 59: 'ù', 60: 'ç', 61: 'e', 62: 'D', 63: 'H', 64: '.', 65: 'L', 66: 't', 67: 'Q', 68: 'q', 69: 'i', 70: 'î'}
{'f': 0, ':'

## 2. Setup a character-level recurrent neural network

**(Question)** Setup a simple embedding layer with `nn.Embedding` to project character indices to `embedding_dim` dimensional vectors. Explain precisely how this layer works and what are its outputs for a given input sequence.

In [4]:
# YOUR CODE HERE

vocab_size = len(vocabulary)

embedding_dim = 5 # ?

embedding_layer = nn.Embedding(vocab_size, embedding_dim)

In [5]:
#Test 

sample = text_data[:10]

sample_tensor = text_to_tensor(sample, ctoi)

s_embedding = embedding_layer(sample_tensor)

print(s_embedding)

tensor([[-0.2596,  0.3374,  0.4850, -0.1812, -0.1988],
        [ 0.6546, -0.9711,  1.0207,  0.2240,  0.4778],
        [ 0.0296,  0.7040,  1.1052,  1.2758,  1.9500],
        [ 0.1543,  0.2170,  1.6953,  0.5396, -0.9453],
        [ 0.1706, -0.8863,  1.4311, -0.9945,  0.9207],
        [-0.2596,  0.3374,  0.4850, -0.1812, -0.1988],
        [ 0.6546, -0.9711,  1.0207,  0.2240,  0.4778],
        [ 0.4482, -0.7093, -0.8015,  2.2838, -0.2236],
        [-1.2021, -0.4061, -1.7473, -0.6024,  0.1015],
        [ 0.0296,  0.7040,  1.1052,  1.2758,  1.9500]],
       grad_fn=<EmbeddingBackward0>)


YOUR ANSWER HERE

**(Question)** Setup a single-layer RNN with `nn.RNN` (without defining a custom class). Use `hidden_dim` size for hidden states. Explain precisely the outputs of this layer for a given input sequence.

In [6]:
# YOUR CODE HERE

hidden_dim = 3

simple_rnn = nn.Sequential(
    embedding_layer,  
    nn.RNN(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True)
)

In [7]:
sample_output = simple_rnn(text_to_tensor(sample, ctoi))
print(sample_output)

(tensor([[ 0.4322,  0.5634,  0.4759],
        [ 0.8594,  0.1568,  0.1570],
        [ 0.8863,  0.7481,  0.4981],
        [ 0.7715,  0.1446,  0.8712],
        [ 0.9039,  0.5293,  0.4650],
        [ 0.5312, -0.0946,  0.7320],
        [ 0.9050,  0.4204,  0.0983],
        [ 0.7283, -0.8322, -0.3519],
        [ 0.1983, -0.5855, -0.4014],
        [ 0.8733,  0.9342,  0.1679]], grad_fn=<SqueezeBackward1>), tensor([[0.8733, 0.9342, 0.1679]], grad_fn=<SqueezeBackward1>))


YOUR ANSWER HERE

**(Question)** Create a simple RNN model with a custom `nn.Module` class. It should contain: an embedding layer, a single-layer RNN, and a dense output layer. For each character of the input sequence, the model should predict the probability of the next character. The forward method should return the probabilities for next characters and the corresponding hidden states.
After completing the class, create a model and apply the forward pass on some input text. Understand and explain the results.

*Note:* depending on how you implement the loss function later, it can be convenient to return logits instead of probabilities, i.e. raw values of the output layer before any activation function. 

In [44]:
class CharRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers=1):
        '''Initialize model parameters and layers.'''
        super().__init__()
        # YOUR CODE HERE
        self.vocab_size = vocab_size
        self.emb = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.lin = nn.Linear(hidden_dim, vocab_size)


    def forward(self, tensor_data, hidden_state=None):
        '''Apply the forward pass for some text data already converted to tensor.'''
        # YOUR CODE HERE
        out = self.emb(tensor_data)
        out, hid = self.rnn(out, hidden_state)
        out = self.lin(out)
        
        return out, hid

# Initialize a model and apply the forward pass on some input text
# YOUR CODE HERE

embedding_dim = 5
hidden_dim = 3

model = CharRNN(vocab_size, embedding_dim, hidden_dim)

sample = text_data[:10]

sample_tensor = text_to_tensor(sample, ctoi)

logits, hidden = model(sample_tensor)

probabilities = F.softmax(logits)

/tmp/ipykernel_5296/3011704167.py:35: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probabilities = F.softmax(logits)


YOUR ANSWER HERE

**(Question)** Implement a simple training loop to overfit on a small input sequence. The loss function should be a categorical cross entropy on the predicted characters. Monitor the loss function value over the iterations.

In [50]:
# Sample a small input sequence into tensor `input_seq` and store its corresponding expected sequence into tensor `target_seq`
# YOUR CODE HERE

# Implement a training loop overfitting an input sequence and monitoring the loss function
def train_overfit(model, input_seq, target_seq, n_iters=200, learning_rate=0.2):
    # YOUR CODE HERE
    criterion = nn.CrossEntropyLoss()
    # criterion = nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  

    for i in range(n_iters):
        outputs, _ = model(input_seq)
        loss = criterion(outputs, target_seq)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if((i+1)%10==0):
            print('Epoch [{}/{}], Loss: {:.4f}'.format(i+1, n_iters, loss.item()))

    
# Initialize a model and make it overfit the input sequence
# YOUR CODE HERE

def one_hot(input_seq, target_seq, vocab_size, ctoi):
    one_hot_vec = np.zeros((len(input_seq), vocab_size))
    for i, c in enumerate(input_seq):
        t_index = ctoi[target_seq[i]]
        one_hot_vec[i][t_index] = 1
    return torch.Tensor(one_hot_vec)


input_seq = text_data[:100]
target_seq = text_data[1:101]

target_seq = one_hot(input_seq, target_seq, vocab_size, ctoi)
input_seq = text_to_tensor(input_seq, ctoi)

embedding_dim = 5
hidden_dim = 3

model = CharRNN(vocab_size, embedding_dim, hidden_dim)

train_overfit(model, input_seq, target_seq)

Epoch [10/200], Loss: 3.8754
Epoch [20/200], Loss: 3.3024
Epoch [30/200], Loss: 2.9640
Epoch [40/200], Loss: 2.7436
Epoch [50/200], Loss: 2.6001
Epoch [60/200], Loss: 2.5165
Epoch [70/200], Loss: 2.4532
Epoch [80/200], Loss: 2.3993
Epoch [90/200], Loss: 2.3563
Epoch [100/200], Loss: 2.3209
Epoch [110/200], Loss: 2.2844
Epoch [120/200], Loss: 2.2483
Epoch [130/200], Loss: 2.2125
Epoch [140/200], Loss: 2.1770
Epoch [150/200], Loss: 2.1422
Epoch [160/200], Loss: 2.1098
Epoch [170/200], Loss: 2.0773
Epoch [180/200], Loss: 2.0372
Epoch [190/200], Loss: 2.0086
Epoch [200/200], Loss: 1.9717


**(Question)** Implement a `predict_argmax` method for your `RNN` model. Then, verify your overfitting: use some characters of your input sequence as context to predict the remaining ones. Experiment with the current model and analyze the results.

In [58]:
class CharRNN(CharRNN):
    def predict_argmax(self, context_tensor, n_predictions):
        # Apply the forward pass for the context tensor
        # Then, store the last prediction and last hidden state
        # YOUR CODE HERE
        out, hid = self.forward(context_tensor)
        self.last_out = out[-1]
        self.last_hid = hid[-1].unsqueeze(-1)
        
        # Use the last prediction and last hidden state as inputs to the next forward pass
        # Do this in a loop to predict the next `n_predictions` characters
        # YOUR CODE HERE
        predictions = []
        for i in range(n_predictions):
            str_out = itoc[np.argmax(self.last_out.detach().numpy())]
            self.last_out = text_to_tensor(str_out, ctoi)
            log, hid = self.forward(self.last_out, self.last_hid)
            self.last_out = log
            self.last_hid = hid.unsqueeze(-1)
            new_str = itoc[np.argmax(self.last_out)]
            predictions.append(new_str)
            
            
        return predictions
            
        

# Initialize a model and make it overfit as above
# Then, verify your overfitting by predicting characters given some context
# YOUR CODE HERE

input_seq = text_data[:100]
target_seq = text_data[1:101]

target_seq = one_hot(input_seq, target_seq, vocab_size, ctoi)
input_seq = text_to_tensor(input_seq, ctoi)

embedding_dim = 5
hidden_dim = 3

model = CharRNN(vocab_size, embedding_dim, hidden_dim)

train_overfit(model, input_seq, target_seq)

predictions = model.predict_argmax(text_to_tensor("Les fleurs", ctoi), 50)



Epoch [10/200], Loss: 3.8667
Epoch [20/200], Loss: 3.3059
Epoch [30/200], Loss: 2.9643
Epoch [40/200], Loss: 2.7557
Epoch [50/200], Loss: 2.6057
Epoch [60/200], Loss: 2.4850
Epoch [70/200], Loss: 2.3951
Epoch [80/200], Loss: 2.3280
Epoch [90/200], Loss: 2.2766
Epoch [100/200], Loss: 2.2343
Epoch [110/200], Loss: 2.1935
Epoch [120/200], Loss: 2.1579
Epoch [130/200], Loss: 2.1258
Epoch [140/200], Loss: 2.0966
Epoch [150/200], Loss: 2.0702
Epoch [160/200], Loss: 2.0473
Epoch [170/200], Loss: 2.0276
Epoch [180/200], Loss: 2.0103
Epoch [190/200], Loss: 1.9946
Epoch [200/200], Loss: 1.9806


RuntimeError: For unbatched 2-D input, hx should also be 2-D but got 1-D tensor

YOUR ANSWER HERE

Using the argmax function to predict the next character can yield a deterministic generator always predicting the same characters. Instead, it is common to predict the next character by sampling from the distribution of output predictions, adding some randomness into the generator.

**(Question)** Implement a `predict_proba` method for your `RNN` model. It should be very similar to `predict_argmax`, but instead of using argmax, it should randomly sample from the output predictions. To do that, you can use the `torch.distribution.Categorical` class and its `sample()` method. Verify that your method correctly added some randomness.

In [ ]:
class CharRNN(CharRNN):
    def predict_proba(self, input_context, n_predictions):
        # YOUR CODE HERE
        raise NotImplementedError()

# Verify that your predictions are not deterministic anymore
# YOUR CODE HERE
raise NotImplementedError()

## 3. Train the RNN model on text data

**(Question)** Adapt your previous code to implement a proper training loop for a text dataset. To do so, we need to specify a sequence length `seq_len`, acting similarly to the batch size in classic neural networks. Then, you can either randomly sample sequences of length `seq_len` from the text dataset over `n_iters` iterations, or properly loop over the text dataset for `n_epochs` epochs (with a random starting point for each epoch to ensure different sequences), to make sure the whole dataset is seen by the model. Feel free to adjust training and model parameters empirically. Start with a small model and a small subset of the text dataset, then move on to larger experiments. Remember to use GPU if available.

In [ ]:
# Create the text dataset, compute its mappings and convert it to tensor
# YOUR CODE HERE
raise NotImplementedError()

# Initialize training parameters
# YOUR CODE HERE
raise NotImplementedError()

# Initialize a character-level RNN model
# YOUR CODE HERE
raise NotImplementedError()

# Setup the training loop
# Regularly record the loss and sample from the model to monitor what is happening
# YOUR CODE HERE
raise NotImplementedError()

**(Question)** From your trained model, play around with its predictions: start with a custom input sequence and ask the model to predict the rest. Analyze and comment your results.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

## 4. Experiment with different RNN architectures

**(Question)** Experiment with different RNN architecures. Potential ideas are multi-layer RNNs, GRUs and LSTMs. All models can be extended to multi-layer using the `num_layers` parameter. Analyze and comment your results.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE